In [ ]:
import tensorflow as tf
import keras
import random as python_random
from tensorflow.keras.layers import Input, Embedding, Dropout, Dense, Flatten, Concatenate, BatchNormalization, LeakyReLU, Multiply
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.activations import relu, gelu
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

import random
from tqdm import tqdm
tqdm.pandas()

In [ ]:
# GPU 사용
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
def reset_random_seeds():
    tf.random.set_seed(42)
    np.random.seed(42)
    python_random.seed(42)
reset_random_seeds()

In [ ]:
base_df = pd.read_pickle("C:\\Users\\Desktop\\final_JEJ_Cell_Phones_and_Accessories.pkl")
base_df

In [ ]:
df = base_df

In [ ]:
# Encode user and item IDs
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

df['reviewerID'] = user_encoder.fit_transform(df['reviewerID'])
df['asin'] = item_encoder.fit_transform(df['asin'])

In [ ]:
# Count unique users and items
user_num = df['reviewerID'].nunique()
item_num = df['asin'].nunique()

print(f'Unique Users: {user_num}')
print(f'Unique Items: {item_num}')

In [ ]:
# Split the dataset
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Training dataset
train_users, train_items = train_data[['reviewerID']].values, train_data[['asin']].values
train_ratings = train_data['overall'].values
train_image = np.array(train_data['VGG16'].tolist())

# Test dataset
test_users, test_items = test_data[['reviewerID']].values, test_data[['asin']].values
test_ratings = test_data['overall'].values
test_image = np.array(test_data['VGG16'].tolist())

In [ ]:
def UCAM_IMAGE(user_num, item_num, feature_dims, learning_rate, dropout_rate):

    # user
    user_input = Input(shape=(1,), dtype='int32', name='UserInput')
    user_embedding = Embedding(user_num, feature_dims, input_length=user_input.shape[1], name='UserIDEmb')(user_input)
    user_embedding = Flatten(name='UserFlatten')(user_embedding)

    # item
    item_input = Input(shape=(1,), dtype='int32', name='ItemInput')
    item_embedding = Embedding(item_num, feature_dims, input_length=item_input.shape[1], name='ItemIDEmb')(item_input)
    item_embedding = Flatten(name='itemFlatten')(item_embedding)

    # GMF Layer
    GMF = Multiply()([user_embedding, item_embedding])
        
    # MLP Layer
    MLP_input = Concatenate(name='UserItemLayer')([user_embedding, item_embedding])
    MLP_input_dense = Dense(units=256, activation='relu', name='Dense_MLP_1')(MLP_input)
    MLP_input_dense = Dropout(rate=dropout_rate)(MLP_input_dense)    
    
    # Image Input and MLP Layers
    input_image = Input(shape=(4096,), name='ImageInput')
    input_image_dense = Dense(2048, activation='relu', name='Dense_Image_1')(input_image)
    input_image_dense = Dense(1024, activation='relu', name='Dense_Image_2')(input_image_dense)
    input_image_dense = Dense(256, activation='relu', name='Dense_Image_3')(input_image_dense)
    
    # Concatenate GMF-MLP and image
    GMF_MLP_Image = Concatenate(name='FinalLayer')([GMF, MLP_input_dense, input_image_dense])

    GMF_MLP_Image_dense = Dense(units=256, activation='relu', name='GMF_MLP_Image_1')(GMF_MLP_Image)
    GMF_MLP_Image_dense = Dropout(rate=dropout_rate)(GMF_MLP_Image_dense)
    GMF_MLP_Image_dense = Dense(units=128, activation='relu', name='GMF_MLP_Image_2')(GMF_MLP_Image_dense)
    GMF_MLP_Image_dense = Dropout(rate=dropout_rate)(GMF_MLP_Image_dense)
    GMF_MLP_Image_dense = Dense(units=64, activation='relu', name='GMF_MLP_Image_3')(GMF_MLP_Image_dense)
    GMF_MLP_Image_dense = Dropout(rate=dropout_rate)(GMF_MLP_Image_dense)
    GMF_MLP_Image_dense = Dense(units=16, activation='relu', name='GMF_MLP_Image_4')(GMF_MLP_Image_dense)
    GMF_MLP_Image_dense = Dropout(rate=dropout_rate)(GMF_MLP_Image_dense)
    
    # Compile Model
    outputs = Dense(1, activation='linear', name='outputs')(GMF_MLP_Image_dense)

    # Model
    model = Model(inputs=[user_input, item_input, input_image], outputs=outputs)
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate), loss = "mse", metrics = ["mse", "mae"])

    return model 

In [ ]:
# Configuration of EarlyStopping
es = EarlyStopping(monitor='val_loss', mode = 'min', verbose = 1, patience = 5, restore_best_weights = True)

batch size of [128, 256, 512, 1024] and
the learning rate of [0.0001 ,0.0005, 0.001, 0.005].

In [ ]:
# Parameter setting
dropout_rate = 0.2
feature_dims = 256
learning_rate = 0.001
batch_size=128 

proposed_model = UCAM_IMAGE(user_num, item_num, feature_dims, learning_rate, dropout_rate)

In [ ]:
proposed_model.fit([train_users, train_items, train_image], train_ratings, batch_size=batch_size, epochs=100, validation_split=0.125,callbacks=[es])

In [ ]:
# Evaluate the model
predicted_ratings = proposed_model.predict([test_users, test_items, test_image])

# Calculate MAE
MAE = mean_absolute_error(test_ratings, predicted_ratings)
print(f"MAE: {MAE:.3f}")

# Calculate MSE
MSE = mean_squared_error(test_ratings, predicted_ratings)
print(f"MSE: {MSE:.3f}")

# Calculate RMSE
RMSE = np.sqrt(MSE)
print(f"RMSE: {RMSE:.3f}")

# Calculate MAPE
MAPE = mean_absolute_percentage_error(test_ratings, predicted_ratings) * 100
print(f"MAPE: {MAPE:.3f}%")

In [ ]:
print( MAE, "\n", MSE, "\n", RMSE, "\n", MAPE )